# Forming Queries through the API!

Get familiar with the tools available for querying the database. The simplest way is to use the api classes 
* [`snowexsql.api.PointMeasurements`](https://github.com/SnowEx/snowexsql/blob/830fa76de8cf13c5101e1b4b663c1b399f81d7e6/snowexsql/api.py#L185)
* [`snowexsql.api.LayerMeasurements`](https://github.com/SnowEx/snowexsql/blob/830fa76de8cf13c5101e1b4b663c1b399f81d7e6/snowexsql/api.py#L262)

* Each class has to very useful functions
  1. [`from_filter`](https://github.com/SnowEx/snowexsql/blob/830fa76de8cf13c5101e1b4b663c1b399f81d7e6/snowexsql/api.py#L192)
  2. [`from_area`](https://github.com/SnowEx/snowexsql/blob/830fa76de8cf13c5101e1b4b663c1b399f81d7e6/snowexsql/api.py#L210)

## Useful Function - `from_filter`

Use the from filter function to find density profiles


In [ ]:
# Import in our two classes to access the db
from snowexsql.api import LayerMeasurements
from datetime import datetime 

# Find some density pit measurements at the Boise site in december 2019.
df = LayerMeasurements.from_filter(
    type="density",
    site_name="Boise River Basin",
    date_less_equal=datetime(2020, 1, 1),
    date_greater_equal=datetime(2019, 12, 1),
)

# Plot Example!
df.plot()

# Show off the dataframe
df

# Analysis Example - Find the bulk density 
df['value'] = df['value'].astype(float)
print(df[['site_id', 'value']].groupby(by='site_id').mean())

## Useful Function - `from_area`
Find specific surface area within a certain distance of a pit.

In [ ]:
# Import our api class
from snowexsql.api import LayerMeasurements
from datetime import datetime
import geopandas as gpd 

# import some gis functionality 
from shapely.geometry import Point 

# Find some SSA measurements within a distance of a known point
pnt = Point(740820.624625,4.327326e+06)
df = LayerMeasurements.from_area(pt=pnt, crs=26912, buffer=500,
    type='specific_surface_area')

# plot up the results
ax = df.plot()

# plot the site so we can see how close everything is.
site = gpd.GeoDataFrame(geometry=[pnt], crs=26912)
site.plot(ax=ax, marker='^', color='magenta')

# show off the dataframe
df

## How do I know what to filter on?
We got tools for that! Each class has a host of functions that start with `all_*` these function return the unique value in that column. 

 * `all_types` - all the data types e.g. depth, swe, density...
 * `all_instruments` - all instruments available in the table
 * `all_dates` - all dates listed in the table
 * `all_site_names` - all the site names available in the table. e.g. Grand Mesa

In [ ]:
from snowexsql.api import PointMeasurements

# Instatiate the class to use the properties!
measurements = PointMeasurements()

# Get the unique data names/types in the table
results = measurements.all_types
print('Available types = {}'.format(', '.join([str(r) for r in results])))

# Get the unique instrument in the table
results = measurements.all_instruments
print('\nAvailable Instruments = {}'.format(', '.join([str(r) for r in results])))

# Get the unique dates in the table
results = measurements.all_dates
print('\nAvailable Dates = {}'.format(', '.join([str(r) for r in results])))

# Get the unique site names in the table
results = measurements.all_site_names
print('\nAvailable sites = {}'.format(', '.join([str(r) for r in results])))

### More specific filtering options
Sometimes we need a bit more filtering to know more about what I can filter on. Questions like "What dates was the SMP used?" are a bit more complicated than "Give me all the dates for snowex"

The good news is, we have tool for that! `from_unique_entries` is your friend!

In [ ]:
# import layer measurements
from snowexsql.api import LayerMeasurements

# Query dates where SMP was used
LayerMeasurements.from_unique_entries(['date'], instrument='snowmicropen')

## Query Nuances
### Limit size 
To avoid accidental large queries, we have added some bumper rails. By default if you ask for more than 1000 records then an error will pop up unless you explicitly say you want more. 

**Try This**: Do a large query. Run the code block below without the limit keyword argument ("kwarg"):

In [ ]:
# Import PointMeasurements
from snowexsql.api import PointMeasurements

# Query db using a vague filter or on a huge dataset like GPR but remove the limit kwarg
df = PointMeasurements.from_filter(type='two_way_travel', limit=100)

# Show the dataframe
df



We have added this on the db to allow you to explore without accidentally pulling the entire SnowEx universe down. If you know you want a large query (defined as > 1000) then use the `limit = ####` option in the `from_filter` or `from_area` function.

**Warning** - It is better to filter using other things besides the limit because the limit is not intelligent. It will simply limit the query by the order of entries that were submitted AND fits your filter. So if you encounter this then consider how to tighten up the filter.

### List of Criteria
You can use lists in your requests too!

In [ ]:
# Import layer measurements
from snowexsql.api import LayerMeasurements

# Grab all the data that used the one of these instruments (hint hint SSA)
ssa_instruments = ["IS3-SP-15-01US", "IRIS",  "IS3-SP-11-01F"]

# Query the DB (throw a limit for safety)
LayerMeasurements.from_filter(instrument=ssa_instruments, limit=100)

### Greater than or Less than
Sometimes we want to isolate certain ranges of value or even dates. The `greater_equal` and `less_equal` terms can be added on to `value` or `dates`. 

* `date_greater_equal`
* `date_less_equal`
* `value_greater_equal`
* `value_less_equal`
 

In [ ]:
# Import the point measurements class
from snowexsql.api import PointMeasurements

# Filter values > 100 cm from the pulse ecko GPR
df = PointMeasurements.from_filter(value_greater_equal=100, type='depth', instrument='pulse EKKO Pro multi-polarization 1 GHz GPR', limit=100)

# Show off the dataframe
df

## Recap 
You just came in contact with the new API tools. We can use each API class to pull from specific tables and filter the data. 
**You should know:**
* How to build queries using `from_filter`, `from_area`, `from_unique_entries`
* Determine what values to filter on
* Manage the limit error
* Filtering on greater and less than
  
If you don't feel comfortable with these, you are probably not alone, let's discuss it!